In [ ]:
# default_exp att_stats

# AttStats

> Class representing attribute statistics.


Attribute statistics consist of the average confidence for the attribute and the average confidence for each of it's values.

**Example:**

"@attribute outlook {sunny, overcast, rainy}
<br> @attribute temperature {hot, mild, cool}
<br>
<br> @data
<br> sunny[0.5];rainy[0.3];overcast[0.2],hot
<br> sunny[0.1];rainy[0.4];overcast[0.5],mild
<br> sunny[0.7];rainy[0.1];overcast[0.2],cool"

Statistics for "outlook" attribute are as follows:
 * average confidence: ( max(0.5, 0.3, 0.2) + max(0.1, 0.4, 0.5) + max(0.7, 0.1, 0.2) ) / 3 = 0.57
 * statistics: [(0.5 + 0.1 + 0.7) / 3, (0.3 + 0.4 + 0.1) / 3, (0.2 + 0.5 + 0.2) / 3] = [0.43, 0.27, 0.3]

In [ ]:
# export
from typing import List,Dict
import pandas as pd
import numpy as np
import warnings


from .value import Value
from .attribute import Attribute
# from pyuid3.data import Data   # may cause problems

In [ ]:
class AttStats:
    def __init__(self, statistics: Dict[str,Value], avg_confidence: float, avg_abs_importance: float, total_samples: int,  att_type: int):
        self.statistics = statistics
        self.avg_confidence = avg_confidence
        self.avg_abs_importance = avg_abs_importance
        self.att_type = att_type
        self.total_samples = total_samples

    @staticmethod
    def calculate_statistics(att: Attribute, data: 'Data') -> 'AttStats':    # TODO: rename to get_stats
        conf_sum = {}
        avg_conf = 0
        avg_abs_importance = 0

        instances = data.instances #get_instances() #no_need2copy
        
        if not instances:
            return AttStats(conf_sum, avg_conf, avg_abs_importance,0, att.get_type())
        
        att_name=att.get_name()
        for instance in instances:
            r = instance.get_reading_for_attribute(att_name)
            values = r.get_values()
            for v in values:
                valname = v.get_name()
                old = conf_sum.get(valname,None)
                if old is not None: 
                    conf_sum[valname] +=  v.get_confidence()
                else:
                    conf_sum[valname] = v.get_confidence()
            
            avg_conf += r.get_most_probable().get_confidence()
            avg_abs_importance += sum(abs(iv) for iv in r.get_most_probable().get_importances().values())

        size = len(data)
        avg_conf /= size
        avg_abs_importance /= size
        stats = {}
        for stat_k,stat_v in conf_sum.items():
        #     #Walkaround to deal with numerical values that can have decimal places, e.g.to make sure  3 == 3.0
            if att.get_type() == Attribute.TYPE_NUMERICAL:
                stats[str(float(stat_k))]=(Value(stat_k, stat_v/size))
            else:
                stats[stat_k]=(Value(stat_k, stat_v/size))
        return AttStats(stats, avg_conf,avg_abs_importance=avg_abs_importance, total_samples=size, att_type=att.get_type())




    def get_statistics(self) -> List[Value]: 
        return list(self.statistics.values())

    def get_avg_confidence(self) -> float:
        return self.avg_confidence

    def get_avg_abs_importance(self) -> float:
        return self.avg_abs_importance

    
    def get_stat_for_value(self, value_name: str) -> float:
        #Walkaround in case of numerical values having decimal places, e.g.to make sure  3 == 3.0
        if self.att_type == Attribute.TYPE_NUMERICAL:
            warnings.warn("Warning: calculating confidence for contibues value. Consider using get_total_stat_for_lt_value or get_total_stat_for_gte_value")
            value_name = str(float(value_name))
        if value_name in self.statistics.keys():
            return self.statistics[value_name].get_confidence()
        else:
            return 0
        
    def get_stat_for_lt_value(self, value_name: str) -> float:        
        return np.sum([c.get_confidence() for v,c in self.statistics.items() if float(value_name) > float(v)])/self.total_samples
    
    def get_stat_for_gte_value(self, value_name: str) -> float:        
        return np.sum([c.get_confidence() for v,c in self.statistics.items() if float(value_name) <= float(v)])/self.total_samples
        

    def get_most_probable(self) -> Value:
        statistics = list(self.statistics.values())
        confidence = [value.get_confidence() for value in statistics]
        highest_conf = max(confidence)
        index = confidence.index(highest_conf)
        return  statistics[index]
   

    def __str__(self) -> str:
        result = '{'
        for value in self.statistics.values():
            result += str(value) + ','
        result = result[:-1]  # delete the last coma ','
        result += '}'
        return result
    
    def copy(self):
        return type(self)(statistics=self.statistics.copy(), avg_confidence=self.avg_confidence ,avg_abs_importance=self.avg_abs_importance,att_type=self.att_type , total_samples=self.total_samples )


## Examples

In [ ]:
import math
from pyuid3.data import Data

data = Data.parse_uarff("../resources/weather.nominal.uncertain.arff")
attribute = Attribute('outlook', {'sunny', 'rainy', 'overcast'})
att_stats = AttStats.get_statistics(attribute, data)
assert att_stats.get_avg_confidence() == 0.9428571428571428

vals = att_stats.het_statistics()
print(vals[0])
print(vals[1])
print(vals[2])

rainy[0.39]
overcast[0.24]
sunny[0.37]
